# Capítulo 7 - Pré-processamento dos dados

Nesse capítulo, exploraremos os passos comuns de pré-processamento de dados com os dados a seguir:

In [1]:
#!pip install pyjanitor

In [2]:
# Importar as bibliotecas do capítulo 
import pandas as pd
import janitor as jn

In [3]:
X2 = pd.DataFrame(
    {
        "a":range(5),
        "b": [-100, -50, 0, 200, 1000]
    }
)
X2

,a,b
0,0,-100
1,1,-50
2,2,0
3,3,200
4,4,1000


### Padronize os dados

Alguns algoritmos, como o SVM, apresentam melhor desempenho quando os dados estão padronizados. Cada coluna deve ter um valor de média igual a zero e um desvio-padrão igual a 1. O sklearn disponibiliza um método ".fit_transform" que combina ".fit" e ".transform":

In [4]:
from sklearn import preprocessing
std = preprocessing.StandardScaler()
std.fit_transform(X2)

array([[-1.41421356, -0.75995002],
       [-0.70710678, -0.63737744],
       [ 0.        , -0.51480485],
       [ 0.70710678, -0.02451452],
       [ 1.41421356,  1.93664683]])

Após a adequação dos dados, há diversos atributos que podemos inspecionar:

In [5]:
std.scale_

array([  1.41421356, 407.92156109])

In [6]:
std.mean_

array([  2., 210.])

In [7]:
std.var_

array([2.000e+00, 1.664e+05])

A seguir, apresentamos uma versão com o pandas. Lembrando de que deverá manter o controle da média e do desvio-padrão originais caso use esse código no pré-processamento. Qualquer amostra que vá usar para predições mais tarde deverá ser padronizada com esses mesmos valores:

In [8]:
X_std = (X2 - X2.mean()) / X2.std()
X_std

,a,b
0,-1.264911,-0.679720
1,-0.632456,-0.570088
2,0.000000,-0.460455
3,0.632456,-0.021926
4,1.264911,1.732190


In [9]:
X_std.mean()

a    4.440892e-17
b    0.000000e+00
dtype: float64

In [10]:
X_std.std()

a    1.0
b    1.0
dtype: float64

### Escale para un intervalo

Escalar para um intervalo consiste em traduzir os dados de modo que estejam entre 0 e 1, inclusive. Ter os dados limitados pode ser conveniente. No entanto, se houver valores discrepantes, tome cuidado ao usar esse recurso.

In [11]:
from sklearn import preprocessing
mms = preprocessing.MinMaxScaler()
mms.fit(X2)
mms.transform(X2)

array([[0.        , 0.        ],
       [0.25      , 0.04545455],
       [0.5       , 0.09090909],
       [0.75      , 0.27272727],
       [1.        , 1.        ]])

Eis uma versão com o pandas

In [12]:
(X2 - X2.min()) / (X2.max() - X2.min())

,a,b
0,0.00,0.000000
1,0.25,0.045455
2,0.50,0.090909
3,0.75,0.272727
4,1.00,1.000000


### Variáveis dummy

Podemos usar o pandas para criar variáveis dummy a partir de dados de categoria. Esse procedimento é também conhecido como codificação one-hot (one-hot encoding) ou codificação de indicadores (indicator encoding). Variáveis dummy são particularmente úteis caso os dados sejam nominais (não ordenados). A função "get_dummies" do pandas cria várias colunas para uma coluna de categorias, cada uma contendo 1 ou 0 de acordo com o fato de a coluna original ter a respectiva categoria

In [13]:
X_cat = pd.DataFrame(
    {
        "name":["George", "Paul"],
        "inst": ["Bass", "Guitar"],
    }
)
X_cat

,name,inst
0,George,Bass
1,Paul,Guitar


A seguir apresentamos uma versão com o pandas. Observe que a opção "drop_first" pode ser usada para eliminar uma coluna (uma das colunas dummy é uma combinação linear das demais colunas):

In [14]:
pd.get_dummies(X_cat, drop_first=True)

,name_Paul,inst_Guitar
0,0,0
1,1,1


A biblioteca pyjanitor também pode separar colunas com a função "expand_column":

In [15]:
X_cat2 = pd.DataFrame(
    {
        "A":[1, None, 3],
        "names": [
            "Fred, George",
            "George",
            "John, Paul",
        ],
    }
)
jn.expand_column(X_cat2, "names", sep=",")

,A,names,George,Paul,Fred,George,John
0,1.0,"Fred, George",1,0,1,0,0
1,NaN,George,0,0,0,1,0
2,3.0,"John, Paul",0,1,0,0,1


Se tivermos dados nominais com alta cardinalidade, podemos usar uma codificação de rótulos(label encoding).

### Codificador de rótulos

Uma alternativa à codificação de variáveis dummy é a codificação de rótulos. Nesse caso, cada dado de categoria será atribuído a um número.É um método conveniente para dados com alta cardinalidade. Esse codificador impõe uma ordem, que poderá ser ou não desejável. Menos espaço poderá ser ocupado em comparação com uma codificação one-hot, e alguns algoritmos (de árvores) são capazes de lidar com essa codificação.

O codificador de rótulos consegue lidar somente com uma coluna de cada vez:

In [16]:
from sklearn import preprocessing
lab = preprocessing.LabelEncoder()
lab.fit_transform(X_cat.name)

array([0, 1])

Se tiver valores modificados, aplique o método ".inverse_transform" para decodificá-los:

In [17]:
lab.inverse_transform([1, 1, 0])

array(['Paul', 'Paul', 'George'], dtype=object)

O pandas também pode ser usado para uma codificação de rótulos. Inicialmente, deve converter a coluna em um tipo para categorias e, em seguida, extrair daí o código numérico:
O código a seguir criará uma nova série de dados numéricos a partir de uma série do pandas. Usaremos o método ".as_ordered" para garantir que a categoria esteja ordenada:

In [18]:
X_cat.name.astype(
    "category"
).cat.as_ordered().cat.codes + 1

0    1
1    2
dtype: int8

### Codificação de frequência 

Outra opção para lidar com dados de categoria com alta cardinalidade é a codificação de frequência (frequency encoding). Isso significa substituir o nome da categoria pelo contador que ela tinha nos dados de treinamento. Será usado o pandas para essa tarefa.
Inicialmente, será utilizado o método ".value_counts" do pandas para fazer um mapeamento (uma série do pandas que mapeia strings e contadores). Com o mapeamento, podemos usar o método ".map" para fazer a codificação:

In [19]:
mapping = X_cat.name.value_counts()
X_cat.name.map(mapping)

0    1
1    1
Name: name, dtype: int64

Não esquecer de armazenar o mapeamento dos dados de treinamento para que seja possível codificar dados futuros com os mesmos dados.

### Extraindo categorias a partir de strings

Uma forma de aumentar a precisão do modelo de dados do Titanic é extrair os títulos dos nomes. Um truque rápido para encontrar as trincas mais comuns é usar a classe Counter:

In [20]:
# Caminho em Pasta
path = "datasets/titanic/titanic3.xls"
df = pd.read_excel(path)

In [21]:
from collections import Counter
c = Counter()
def triples(val):
    for i in range(len(val)):
        c[val[i : i + 3]] += 1
df.name.apply(triples)
c.most_common(10)

[(', M', 1282),
 (' Mr', 954),
 ('r. ', 830),
 ('Mr.', 757),
 ('s. ', 460),
 ('n, ', 320),
 (' Mi', 283),
 ('iss', 261),
 ('ss.', 261),
 ('Mis', 260)]

Podemos ver que "Mr." e "Miss" são muito comuns.
Outra opção é usar uma expressão regular para extrair a letra maiúscula seguida de letras minúsculas e um ponto:

In [22]:
df.name.str.extract(
    "([A-Za-z]+)\.", expand=False
).head()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
/tmp/ipykernel_12124/178673788.py:2: DeprecationWarning: invalid escape sequence \.
  "([A-Za-z]+)\.", expand=False


0      Miss
1    Master
2      Miss
3        Mr
4       Mrs
Name: name, dtype: object

Podemos usar ".value_counts" para ver a frequência desses títulos:

In [23]:
df.name.str.extract(
    "([A-Za-z]+)\.", expand=False
).value_counts()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
/tmp/ipykernel_12124/532577061.py:2: DeprecationWarning: invalid escape sequence \.
  "([A-Za-z]+)\.", expand=False


Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Ms            2
Major         2
Capt          1
Sir           1
Dona          1
Jonkheer      1
Countess      1
Don           1
Mme           1
Lady          1
Name: name, dtype: int64

Ao usar essas manipulações e o pandas, pode criar variáveis dummy ou combinar colunas com contadores baixos em outras categorias (ou descartá-las).

### Outras codificações de categoria

A biblioteca categorical_encoding é um conjunto de transformadores do scikit-learn, usados para converter dados de categorias em dados numéricos. Um bom recurso dessa biblioteca é que ela gera DataFrames do pandas (de modo diferente do scikit-learn, que transforma os dados em arrays numpy).

Um algoritmo implementado nessa biblioteca é um codificador de hashes. É útil caso não saiba com antecedência quantas categorias há, ou se estiver usando um conjunto de palavras para representar texto. O algoritmo gerará hash de colunas de categorias em n_components. Se estiver usando online learning (aprendizado online), isto é, modelos que podem ser atualizados, esse recurso poderá ser muito conveniente.

In [24]:
#!pip install category_encoders

In [25]:
import category_encoders as ce
he = ce.HashingEncoder(verbose=1)
he.fit_transform(X_cat)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,0,0,1,0,1,0,0
1,0,2,0,0,0,0,0,0


O codificador de ordinais(ordinal encoder) pode converter colunas de categorias que tenham uma ordem em uma única coluna de números. Em nosso exemplo, converteremos a coluna de tamanho (size) em númneros ordinais. Se houver um valor ausente no dicionário de mapeamento, um valor default igual a -1 será usado:

In [26]:
size_df = pd.DataFrame(
    {
        "name": ["Fred", "John", "Matt"],
        "size": ["small", "med", "xxl"]
    }
)

ore = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"size",
            "mapping":{
                "small": 1,
                "med": 2,
                "lg": 3,
            },
        }
    ]
)

ore.fit_transform(size_df)

,name,size
0,Fred,1.0
1,John,2.0
2,Matt,-1.0


A referência em "https://oreil.ly/JUtYh" explica vários algoritmos que estão na biblioteca categorical_encoding.

Se tiver dados com alta cardinalidade (um número grande de valores únicos), considere usar um dos codificadores bayseianos (Bayesian encoders) que geram uma única coluna por coluna de categoria. São eles: "TargetEncoder", "LeaveOneOutEncoder", "WQEncoder", "JamesSteinEncoder" e "MEstimateEncoder".
    
Por exemplo, para converter a coluna survived (sobrevivência) dos dados do Titanic em uma combinação da probabilidade posterior do alvo com a probabilidade anterior dada a informação de título (dado de categoria), utilize o código a seguir:
    


In [27]:
def get_title(df):
    return df.name.str.extract(
        "([A-Za-z]+)\.", expand=False
    )

te = ce.TargetEncoder(cols="Title")
te.fit_transform(
    df.assign(Title=get_title), df.survived
)["Title"].head()

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
/tmp/ipykernel_12124/1773409615.py:3: DeprecationWarning: invalid escape sequence \.
  "([A-Za-z]+)\.", expand=False


0    0.676923
1    0.506139
2    0.676923
3    0.162483
4    0.786802
Name: Title, dtype: float64

### Engenharia de dados para datas

A biblioteca fastai tem uma função "add_datepart", que gerará colunas com atributos de data com base em uma coluna de data e hora. Isso é conveniente, pois a maioria dos algoritmos de machine learning não é capaz de inferir esse tipo de sinal a partir da representação numérica de uma data:

In [28]:
#!pip install fastai

In [29]:
from fastai.tabular.core import(
    add_datepart,
)

dates = pd.DataFrame(
    {
        "A":pd.to_datetime(
            ["9/17/2001", "Jan, 1, 2002"]
        )
    }
)

add_datepart(dates, "A")
dates.T

/home/luciano/anaconda3/lib/python3.9/site-packages/tables/__init__.py:24: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from .utilsextension import (
/home/luciano/anaconda3/lib/python3.9/site-packages/tables/__init__.py:24: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from .utilsextension import (
/home/luciano/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:20: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_numpy_version = LooseVersion('1.9.3')
/home/luciano/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_numexpr_version = LooseVersion('2.6.2')
/home/luciano/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_hdf5_version = Loose

,0,1
AYear,2001,2002
AMonth,9,1
AWeek,38,1
ADay,17,1
ADayofweek,0,1
ADayofyear,260,1
AIs_month_end,False,False
AIs_month_start,False,True
AIs_quarter_end,False,False
AIs_quarter_start,False,True


O add_datepart altera o DataFrame, algo que o pandas pode fazer, mas, em geral, não faz!

### Adição do atributo col_na

A biblioteca fastai costumava ter uma função para criar uma coluna para preenchimento de um valor ausente (com mediana) e indicar que um valor estava ausente. Saber que um valor estava ausente poderia ser usado como uma informação. Eis uma cópia da função e um exemplo de sua utilização:

In [30]:
from pandas.api.types import is_numeric_dtype
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name + "_na"] = pd.isnull(col)
            filler = (
                na_dict[name]
                if name in na_dict
                else col.median()
            )
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict
data = pd.DataFrame({"A": [0, None, 5, 100]})
fix_missing(data, data.A, "A", {})

{'A': 5.0}

In [31]:
data

,A,A_na
0,0.0,False
1,5.0,True
2,5.0,False
3,100.0,False


A seguir, apresentamos uma versão com o pandas:

In [32]:
data = pd.DataFrame({"A": [0, None, 5, 100]})
data["A_na"] = data.A.isnull()
data["A"] = data.A.fillna(data.A.median())

In [33]:
data

,A,A_na
0,0.0,False
1,5.0,True
2,5.0,False
3,100.0,False


### Engenharia de dados manual

O pandas pode ser usado para gerar novos atributos. Para o conjunto de dados do Titanic, podemos agregar dados de cabine(idade máxima por cabine, idade média por cabine etc). Para obter dados agregados por cabine, utilize o método ".groupby" do pandas para criá-los. Em seguida, alinhe-os com os dados originais usando o método ".merge"

In [34]:
agg = (
    df.groupby("cabin")
    .agg("min,max,mean,sum".split(","))
    .reset_index()
)

agg.columns = [ 
    "_".join(c).strip("_")
    for c in agg.columns.values
]
agg_df = df.merge(agg, on="cabin")

/tmp/ipykernel_12124/3089466328.py:2: FutureWarning: ['name', 'sex', 'ticket', 'embarked', 'boat', 'home.dest'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby("cabin")


In [35]:
agg_df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,...,parch_mean,parch_sum,fare_min,fare_max,fare_mean,fare_sum,body_min,body_max,body_mean,body_sum
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,...,0.5,1,211.3375,211.3375,211.3375,422.675,NaN,NaN,NaN,0.0
1,1,1,"Madill, Miss. Georgette Alexandra",female,15.0000,0,1,24160,211.3375,B5,...,0.5,1,211.3375,211.3375,211.3375,422.675,NaN,NaN,NaN,0.0
2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,...,2.0,8,151.5500,151.5500,151.5500,606.200,135.0,135.0,135.0,135.0
3,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,...,2.0,8,151.5500,151.5500,151.5500,606.200,135.0,135.0,135.0,135.0
4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,...,2.0,8,151.5500,151.5500,151.5500,606.200,135.0,135.0,135.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,3,1,"Sandstrom, Mrs. Hjalmar (Agnes Charlotta Bengtsson)",female,24.0000,0,2,PP 9549,16.7000,G6,...,1.2,6,10.4625,16.7000,14.2050,71.025,NaN,NaN,NaN,0.0
291,3,1,"Sandstrom, Miss. Marguerite Rut",female,4.0000,1,1,PP 9549,16.7000,G6,...,1.2,6,10.4625,16.7000,14.2050,71.025,NaN,NaN,NaN,0.0
292,3,0,"Strom, Miss. Telma Matilda",female,2.0000,0,1,347054,10.4625,G6,...,1.2,6,10.4625,16.7000,14.2050,71.025,NaN,NaN,NaN,0.0
293,3,0,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",female,29.0000,1,1,347054,10.4625,G6,...,1.2,6,10.4625,16.7000,14.2050,71.025,NaN,NaN,NaN,0.0


In [ ]:
Se quisesse calcular as colunas "boas" ou "ruins", poderia criar outra coluna que fosse a soma das colunas agregadas(ou usar outra )